In [1]:
import gspread, logging
account = gspread.service_account("credentials.json")
spreadsheet = account.open("FairDivision")
spreadsheet

<Spreadsheet 'FairDivision' id:1tJPV-y-r1TAx5FqbrqecKPJMeKHTtIDeiYck8eLoGKY>

In [2]:
import input
input.logger.addHandler(logging.StreamHandler())
input.logger.setLevel(logging.INFO)

rows = input.read_rows(spreadsheet)
agents, items, entitlement_normalized_preferences  = input.analyze_rows(rows)

Rows: 1000, Cols: 28
items: ['חוץ', 'בטחון', 'אוצר', 'בט"פ', 'משפטים', 'פנים', 'בריאות', 'חינוך']
agents: ['הליכוד', 'הציונות הדתית', 'ש"ס', 'אגודת ישראל']
entitlements: {'הליכוד': 32, 'הציונות הדתית': 14, 'ש"ס': 11, 'אגודת ישראל': 7}, total: 64.000000
raw_preferences: 
	הליכוד:		{'חוץ': 20.0, 'בטחון': 20.0, 'אוצר': 20.0, 'בט"פ': 10.0, 'משפטים': 10.0, 'פנים': 10.0, 'בריאות': 10.0, 'חינוך': 20.0}		120.000000
	הציונות הדתית:		{'חוץ': 10.0, 'בטחון': 20.0, 'אוצר': 10.0, 'בט"פ': 30.0, 'משפטים': 20.0, 'פנים': 10.0, 'בריאות': 20.0, 'חינוך': 20.0}		140.000000
	ש"ס:		{'חוץ': 5.0, 'בטחון': 5.0, 'אוצר': 20.0, 'בט"פ': 5.0, 'משפטים': 10.0, 'פנים': 30.0, 'בריאות': 20.0, 'חינוך': 20.0}		115.000000
	אגודת ישראל:		{'חוץ': 5.0, 'בטחון': 5.0, 'אוצר': 5.0, 'בט"פ': 5.0, 'משפטים': 5.0, 'פנים': 10.0, 'בריאות': 20.0, 'חינוך': 20.0}		75.000000
normalized_preferences: 
	הליכוד:		{'חוץ': 10.667, 'בטחון': 10.667, 'אוצר': 10.667, 'בט"פ': 5.333, 'משפטים': 5.333, 'פנים': 5.333, 'בריאות': 5.333, 'חינוך': 10.667}		64.

In [3]:
import allocate
allocate.logger.addHandler(logging.StreamHandler())
allocate.logger.setLevel(logging.INFO)

map_agent_to_fractions = allocate.allocate(agents, entitlement_normalized_preferences)

leximin allocation: הליכוד gets { 100.0% of חוץ, 100.0% of בטחון, 100.0% of אוצר, 27.602% of משפטים, 4.298% of פנים, 40.412% of בריאות, 100.0% of חינוך} with value 1.45.
הציונות הדתית gets { 100.0% of בט"פ, 72.398% of משפטים} with value 1.45.
ש"ס gets { 95.702% of פנים} with value 1.45.
אגודת ישראל gets { 59.588% of בריאות} with value 1.45.

bounded sharing allocation: הליכוד gets { 100.0% of חוץ, 100.0% of בטחון, 100.0% of אוצר, 27.602% of משפטים, 4.298% of פנים, 40.412% of בריאות, 100.0% of חינוך} with value 1.45.
הציונות הדתית gets { 100.0% of בט"פ, 72.398% of משפטים} with value 1.45.
ש"ס gets { 95.702% of פנים} with value 1.45.
אגודת ישראל gets { 59.588% of בריאות} with value 1.45.

map_agent_to_fractions: {'הליכוד': array([1.   , 1.   , 1.   , 0.   , 0.276, 0.043, 0.404, 1.   ]), 'הציונות הדתית': array([0.   , 0.   , 0.   , 1.   , 0.724, 0.   , 0.   , 0.   ]), 'ש"ס': array([0.   , 0.   , 0.   , 0.   , 0.   , 0.957, 0.   , 0.   ]), 'אגודת ישראל': array([0.   , 0.   , 0.   , 0.   , 

In [7]:
import output
dir(output)


['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'cells',
 'gspread',
 'logger',
 'logging',
 'worksheet']

In [8]:
new_row_count = len(agents)+2
new_col_count = len(items)+5
output_sheet = output.worksheet(spreadsheet, new_row_count, new_col_count)
output_sheet

# input_range = input.range(1, 1, len(rows), len(rows[0]))
# output.update_cells(input_range)

<Worksheet 'output' id:236568254>

In [10]:
output_sheet.clear()

{'spreadsheetId': '1tJPV-y-r1TAx5FqbrqecKPJMeKHTtIDeiYck8eLoGKY',
 'clearedRange': 'output!A1:M1001'}

In [14]:
# Insert headings:
new_cells = output.cells(rows, agents, items, map_agent_to_fractions)
# new_cells = []
# new_cells += [gspread.Cell(i+1, 1, "=input!"+gspread.utils.rowcol_to_a1(i+1,1)) for i in range(len(rows))]  # Copy column 1
# new_cells += [gspread.Cell(i+1, 2, "=input!"+gspread.utils.rowcol_to_a1(i+1,2)) for i in range(len(rows))]  # Copy column 2
output_sheet.update_cells(new_cells, value_input_option='USER_ENTERED')

{'spreadsheetId': '1tJPV-y-r1TAx5FqbrqecKPJMeKHTtIDeiYck8eLoGKY',
 'updatedRange': 'output!A1:B6',
 'updatedRows': 6,
 'updatedColumns': 2,
 'updatedCells': 12}

In [12]:
# Change the cell format:
first_cell = gspread.utils.rowcol_to_a1(2, 3)
last_cell = gspread.utils.rowcol_to_a1(len(agents)+2, len(items)+5)
output_sheet.format(f"{first_cell}:{last_cell}", {"numberFormat": {"type": "PERCENT", "pattern": "##.#%"}})

{'spreadsheetId': '1tJPV-y-r1TAx5FqbrqecKPJMeKHTtIDeiYck8eLoGKY',
 'replies': [{}]}